In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

def get_team_mapping(engine):
    """Get mapping of team abbreviations to team IDs from all_teams_misc_stats"""
    query = "SELECT TEAM_ID, TEAM_ABBREVIATION FROM all_teams_misc_stats"
    with engine.connect() as conn:
        team_mapping = pd.read_sql(query, conn)
    return dict(zip(team_mapping['TEAM_ABBREVIATION'], team_mapping['TEAM_ID']))

def parse_team_from_matchup(matchup, is_home):
    """Extract team abbreviation from MATCHUP string
    Example: 'PHI vs. HOU' or 'PHI @ HOU'"""
    teams = matchup.replace(' vs. ', ' @ ').split(' @ ')
    return teams[0] if is_home else teams[1]

def update_team_ids(engine):
    # Get team ID mapping
    team_mapping = get_team_mapping(engine)
    
    # Get existing game stats
    query = "SELECT * FROM all_player_game_stats"
    with engine.connect() as conn:
        games_df = pd.read_sql(query, conn)
    
    # Parse team from MATCHUP
    games_df['Team_ID'] = games_df.apply(
        lambda row: team_mapping[parse_team_from_matchup(
            row['MATCHUP'], 
            '@' not in row['MATCHUP']  # Check if home game
        )], 
        axis=1
    )
    
    # Update database with team IDs
    with engine.connect() as conn:
        for index, row in games_df.iterrows():
            update_query = text("""
                UPDATE all_player_game_stats
                SET "Team_ID" = :team_id
                WHERE "PLAYER_ID" = :player_id
                AND "GAME_DATE" = :game_date
            """)
            conn.execute(update_query, {
                'team_id': row['Team_ID'],
                'player_id': row['PLAYER_ID'],
                'game_date': row['GAME_DATE']
            })

In [10]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
print(nba_teams)

team_pairs = {}

for team in nba_teams:
    team_pairs[team['abbreviation']] = team['id']
print()
print(team_pairs)

[{'id': 1610612737, 'full_name': 'Atlanta Hawks', 'abbreviation': 'ATL', 'nickname': 'Hawks', 'city': 'Atlanta', 'state': 'Georgia', 'year_founded': 1949}, {'id': 1610612738, 'full_name': 'Boston Celtics', 'abbreviation': 'BOS', 'nickname': 'Celtics', 'city': 'Boston', 'state': 'Massachusetts', 'year_founded': 1946}, {'id': 1610612739, 'full_name': 'Cleveland Cavaliers', 'abbreviation': 'CLE', 'nickname': 'Cavaliers', 'city': 'Cleveland', 'state': 'Ohio', 'year_founded': 1970}, {'id': 1610612740, 'full_name': 'New Orleans Pelicans', 'abbreviation': 'NOP', 'nickname': 'Pelicans', 'city': 'New Orleans', 'state': 'Louisiana', 'year_founded': 2002}, {'id': 1610612741, 'full_name': 'Chicago Bulls', 'abbreviation': 'CHI', 'nickname': 'Bulls', 'city': 'Chicago', 'state': 'Illinois', 'year_founded': 1966}, {'id': 1610612742, 'full_name': 'Dallas Mavericks', 'abbreviation': 'DAL', 'nickname': 'Mavericks', 'city': 'Dallas', 'state': 'Texas', 'year_founded': 1980}, {'id': 1610612743, 'full_name':

In [2]:
from nba_api.stats.static import teams
import pandas as pd
from sqlalchemy import create_engine, text


def parse_team_from_matchup(matchup, is_home):
    """Extract team abbreviation from MATCHUP string
    Example: 'PHI vs. HOU' or 'PHI @ HOU'"""
    teams = matchup.replace(' vs. ', ' @ ').split(' @ ')
    return teams[0] if is_home else teams[1]
    
def update_team_ids(engine):

    team_mapping = {}
    nba_teams = teams.get_teams()
    for team in nba_teams:
        team_mapping[team['abbreviation']] = team['id']

    print("Got team mappings...")
    
    query = "SELECT * FROM all_player_game_stats"
    with engine.connect() as conn:
        print("Fetching games from database...")
        games_df = pd.read_sql(query, conn)

    print(f"Processing {len(games_df)} records...")
    
    games_df['TEAM_ID'] = games_df.apply(  # Changed to TEAM_ID to match column name
        lambda row: team_mapping[parse_team_from_matchup(
            row['MATCHUP'], 
            '@' not in row['MATCHUP']
        )], 
        axis=1
    )

        # Update database with team IDs
    with engine.connect() as conn:
        with conn.begin():
            for index, row in games_df.iterrows():
                update_query = text("""
                    UPDATE all_player_game_stats
                    SET "TEAM_ID" = :team_id
                    WHERE "PLAYER_ID" = :player_id
                    AND "GAME_DATE" = :game_date
                """)
                conn.execute(update_query, {
                    'team_id': row['TEAM_ID'],
                    'player_id': row['PLAYER_ID'],
                    'game_date': row['GAME_DATE']
                })
                if index % 1000 == 0:
                    print(f"Updated {index} records...")
    print("Update complete!")

In [1]:
from sqlalchemy import create_engine, text

from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")

engine = create_engine(DATABASE_URL)

update_team_ids(engine)

NameError: name 'update_team_ids' is not defined

In [6]:
from nba_api.stats.static import teams

nba_teams = teams.get_teams()
print(nba_teams)

[{'id': 1610612737, 'full_name': 'Atlanta Hawks', 'abbreviation': 'ATL', 'nickname': 'Hawks', 'city': 'Atlanta', 'state': 'Georgia', 'year_founded': 1949}, {'id': 1610612738, 'full_name': 'Boston Celtics', 'abbreviation': 'BOS', 'nickname': 'Celtics', 'city': 'Boston', 'state': 'Massachusetts', 'year_founded': 1946}, {'id': 1610612739, 'full_name': 'Cleveland Cavaliers', 'abbreviation': 'CLE', 'nickname': 'Cavaliers', 'city': 'Cleveland', 'state': 'Ohio', 'year_founded': 1970}, {'id': 1610612740, 'full_name': 'New Orleans Pelicans', 'abbreviation': 'NOP', 'nickname': 'Pelicans', 'city': 'New Orleans', 'state': 'Louisiana', 'year_founded': 2002}, {'id': 1610612741, 'full_name': 'Chicago Bulls', 'abbreviation': 'CHI', 'nickname': 'Bulls', 'city': 'Chicago', 'state': 'Illinois', 'year_founded': 1966}, {'id': 1610612742, 'full_name': 'Dallas Mavericks', 'abbreviation': 'DAL', 'nickname': 'Mavericks', 'city': 'Dallas', 'state': 'Texas', 'year_founded': 1980}, {'id': 1610612743, 'full_name':

In [13]:
def update_team_names(engine):

    nba_teams = teams.get_teams()
    team_dict = {team['id']: team['full_name'] for team in nba_teams}
    print("Got teams")
    
    query = "SELECT * FROM all_player_game_stats"
    with engine.connect() as conn:
        print("Fetching games from database...")
        games_df = pd.read_sql(query, conn)

    print(f"Processing {len(games_df)} records...")

    # Update database with team IDs
    with engine.connect() as conn:
        with conn.begin():
            for index, row in games_df.iterrows():
                update_query = text("""
                    UPDATE all_player_game_stats
                    SET "TEAM_NAME" = :team_name
                    WHERE "PLAYER_ID" = :player_id
                    AND "GAME_DATE" = :game_date
                """)
                conn.execute(update_query, {
                    'team_name': team_dict[row['TEAM_ID']],
                    'player_id': row['PLAYER_ID'],
                    'game_date': row['GAME_DATE']
                })
                if index % 1000 == 0:
                    print(f"Updated {index} records...")
    print("Update complete!")

In [ ]:
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os

load_dotenv()

DATABASE_URL = os.getenv("DATABASE_URL")

engine = create_engine(DATABASE_URL)

update_team_names(engine)

Got teams
Fetching games from database...
Processing 17388 records...
Updated 0 records...
Updated 1000 records...
Updated 2000 records...
Updated 3000 records...
Updated 4000 records...
Updated 5000 records...
Updated 6000 records...
Updated 7000 records...
Updated 8000 records...
Updated 9000 records...
Updated 10000 records...
Updated 11000 records...
Updated 12000 records...
Updated 13000 records...
Updated 14000 records...
Updated 15000 records...
Updated 16000 records...
Updated 17000 records...
Update complete!
